In [42]:
from nltk import word_tokenize
from collections import defaultdict

In [43]:
def count_top_x_words(corpus, top_x, skip_top_n):
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]

In [44]:
def replace_top_x_words_with_vectors(corpus, top_x):
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
        [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
        for s in corpus
    ], topx_dict

In [45]:
def filter_to_top_x(corpus, n_top, skip_n_top=0):
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)

In [46]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding

In [ ]:
!pip3 install lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install keras-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package keras-utils


In [ ]:
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

In [50]:
import nltk
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

In [22]:
%cd /content/drive/My\ Drive/Colab Notebooks
!ls

/content/drive/My Drive/Colab Notebooks
'Copy of Untitled43.ipynb'		      Untitled11.ipynb
 day2_shantanu.ipynb			      Untitled12.ipynb
 Day2_SupplimentaryAssignment.ipynb	      Untitled13.ipynb
 deep_learning.ipynb			      Untitled14.ipynb
'Economic Statistics Data Cleaning.ipynb'     Untitled15.ipynb
 exercise1CG.ipynb			      Untitled16.ipynb
 final_merged.ipynb			      Untitled17.ipynb
'gdp Data Cleaning.ipynb'		      Untitled18.ipynb
'gdp_growth Data Cleaning.ipynb'	      Untitled1.c
'gdp_per_capita Data Cleaning.ipynb'	      Untitled1.ipynb
'gdp_per_capita_growth Data Cleaning.ipynb'  'Untitled (2)'
'gdp_ppp Data Cleaning.ipynb'		      Untitled2.ipynb
'gdp_ppp_per_capita Data Cleaning.ipynb'     'Untitled (3)'
 get_top_x_words.ipynb			      Untitled3.ipynb
'Naive Bayes.ipynb'			     'Untitled (4)'
 SE01.ipynb				      Untitled4.ipynb
 SE-1.ipynb				     'Untitled (5)'
'SE-2 (1).ipynb'			      Untitled5.ipynb
 SE-2.ipynb				      Untitled6.ipynb
 SE-3.ipynb				      Untitled7.ipy

In [25]:
%%writefile get_top_xwords

def filter_to_top_x(corpus, n_top, skip_n_top=0):
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)

Writing get_top_xwords


In [26]:
!ls -l get_top_xwords

-rw------- 1 root root 166 Jul 15 09:40 get_top_xwords


In [47]:
df = pd.read_csv('/content/drive/MyDrive/Final_cleaned_data/20000_cleaned_data.csv')

In [48]:
counter = Counter(df['label'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['label'].map(lambda x: x in top_10_varieties)]

In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [54]:
description_list = df['provision'].tolist()
mapped_list, word_list = filter_to_top_x(description_list, 2500, 10)
varietal_list_o = [top_10_varieties[i] for i in df['label'].tolist()]
varietal_list = to_categorical(varietal_list_o)

In [55]:
max_review_length = 150

In [56]:
mapped_list = sequence.pad_sequences(mapped_list, maxlen=max_review_length)
train_x, test_x, train_y, test_y = train_test_split(mapped_list, varietal_list, test_size=0.3)

In [57]:
embedding_vector_length = 64
model = Sequential()

In [64]:
model.add(Embedding(2500, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(max(varietal_list_o) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5, batch_size=64)

Epoch 1/5
23/23 [==============================] - 1s 17ms/step - loss: 2.2515 - accuracy: 0.1669
Epoch 2/5
23/23 [==============================] - 0s 17ms/step - loss: 2.2277 - accuracy: 0.1794
Epoch 3/5
23/23 [==============================] - 0s 17ms/step - loss: 2.2270 - accuracy: 0.1794
Epoch 4/5
23/23 [==============================] - 0s 18ms/step - loss: 2.2276 - accuracy: 0.1794
Epoch 5/5
23/23 [==============================] - 0s 17ms/step - loss: 2.2280 - accuracy: 0.1794


In [65]:
y_score = model.predict(test_x)
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_y[i][j] for j in range(len(y_score[i]))):
        n_right += 1

In [67]:
print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 18.31%
